In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import plotly.express as px
from decouple import config
import os
from IPython.display import IFrame

# ---------------- Pandas settings --------------- #
# Removes rows and columns truncation of '...'
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


# ----------------- Set Mapbox Token --------------- #
MAPBOX_ACCESS_TOKEN=config("MAPBOX_ACCESS_TOKEN")
px.set_mapbox_access_token(MAPBOX_ACCESS_TOKEN)

# Set Data Dir
DATA_DIR="../drive_thru_testing_locations"

**Source: https://covid19.sokat.ai/**

In [2]:
IFrame("https://covid19.sokat.ai/", width="100%", height=400)

In [3]:
def markers_to_dataframe(url):
    """This function takes the https://covid19.sokat.ai/ web map as input to generate a CSV file of drive-thru testing locations. 
    """
    # Download webpage data with requests
    page = requests.get(url)
    # Rarse the webpage data with BeautifulSoup
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Select scripts in HTML file
    scripts = list(soup.find_all('script'))
    
    # Subset for the script with marker data and begin string splitting
    html = str(list(scripts[7])).split('var marker')
    
    # Use for-loops to generate the names, coordinates, and URLs of drive-thru testing locations
    list_of_names = []
    list_of_coords = []
    list_of_urls = []
    
    for i in range(2,142):
        list_of_names.append(html[i].split('target="_blank">')[1].split('</a></div>')[0])
        list_of_coords.append(html[i].split('target="_blank">')[0].split("L.marker(\\n                [")[1].split("]")[0])
        list_of_urls.append(html[i].split('target="_blank">')[0].split('><a href="')[1].split('"')[0])

    # Concatenate lists into a pandas DataFrame
    dataframe = pd.DataFrame([list_of_names, list_of_coords, list_of_urls]).T
    
    # Rename the columns with appropriate titles
    dataframe.columns = ["Name","Coordinate", "URL"]
    
    # Generate Latitude and Longitude Series with string splitting
    dataframe["Latitude"] = dataframe["Coordinate"].str.split(expand=True)[0].str.replace(",","").astype(float)
    dataframe["Longitude"] = dataframe["Coordinate"].str.split(expand=True)[1].astype(float)
    
    # Return results
    return dataframe

In [4]:
df = markers_to_dataframe("https://covid19.sokat.ai/")
df = df[["Name", "URL", "Latitude", "Longitude"]] # added to parse coordinates column out to save space
df.info() # check for data type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       140 non-null    object 
 1   URL        140 non-null    object 
 2   Latitude   140 non-null    float64
 3   Longitude  140 non-null    float64
dtypes: float64(2), object(2)
memory usage: 4.5+ KB


In [5]:
df.isnull().sum() # Check for missing values

Name         0
URL          0
Latitude     0
Longitude    0
dtype: int64

### Display data

In [6]:
df

,Name,URL,Latitude,Longitude
0,collaborative effort between health care prov...,https://www.adn.com/alaska-news/anchorage/2020...,61.182870,-149.837269
1,Church of the Highlands- Grant\'s Mill,https://www.google.com/amp/s/www.wvtm13.com/am...,33.516999,-86.655847
2,UAMS,https://www.thv11.com/mobile/article/news/heal...,34.749005,-92.320096
3,Mayo Clinic Hospital,https://www.12news.com/article/news/health/cor...,33.662111,-111.953513
4,Mayo Clinic,https://www.12news.com/article/news/health/cor...,33.586042,-111.794079
5,Stanford Health Care,http://med.stanford.edu/news/all-news/2020/03/...,37.441515,-122.168679
6,Kaiser Permanente French Campus,https://www.sfgate.com/bayarea/article/coronav...,37.780666,-122.463766
7,Colorado Department of Public Health and Envi...,https://www.denverpost.com/2020/03/11/colorado...,39.716960,-104.894000
8,Denver Coliseum,https://www.colorado.gov/pacific/cdphe/covid-1...,39.780582,-104.971930
9,Hartford Hospital,https://www.courant.com/news/connecticut/hc-ne...,41.754561,-72.679231


### Save dataframe to `csv`

In [7]:
df.to_csv(os.path.join(DATA_DIR, 'us-drive-thru-testing-locations.csv'))

### Plotting Drive-Thru centers via Plotly Express

In [14]:
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", hover_name="Name", zoom=3)
fig.update_layout(mapbox_style="dark")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()